Trends
------

In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib
import matplotlib.pyplot as plt

# % matplotlib inline

In [4]:
font = {'size'   : 10,
        'weight': 'bold'}
matplotlib.rc('font', **font)

In [5]:
from fbprophet import Prophet

In [6]:
def plot_fit_prophet(series, name):
    
    m = Prophet()
    df = pd.DataFrame({'ds':series.index, 'y':series.values})
    # log values
    df['y'] = np.log(df['y'])
    
    m.fit(df);
    future = m.make_future_dataframe(periods=1)
    forecast = m.predict(future)
    
    fig, ax = plt.subplots(figsize=(7, 4))
    m.plot(forecast, ax=ax);
    ax.set_title(name, fontsize=10, fontweight='bold')
    ax.set_ylabel(ylabel=r'$\log($number of articles$)$', fontsize=10)
    ax.set_xlabel(xlabel=r'years', fontsize=10)
    plt.xticks(rotation=90)
    return plt.savefig('/home/nightwing/rsc/Literature-Article/assets/images/trend_{}.pdf'.format(name),
                        bbox_inches='tight')

In [7]:
df = pd.read_json('../data/data_dec_2017_clean.json')

In [8]:
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y')

Trends
----------------

In [9]:
# the series
yrl = df.groupby(['unique_key', 'date']).size().reset_index().groupby('date').size()

In [10]:
yrl.head(3)

date
1944-01-01    1
1950-01-01    1
1958-01-01    1
dtype: int64

In [12]:
plot_fit_prophet(yrl, 'Full data')

INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/nightwing/anaconda3/envs/literature/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [13]:
yrl_pvn = df.groupby(['unique_key','provenance', 'date']).size().reset_index().groupby(['provenance', 'date']).size()

In [44]:
for api in ['IEEE', 'Nature', 'PLOS', 'Springer', 'arXiv']:
    plot_fit_prophet(yrl_pvn[api], api)

INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
/home/nightwing/anaconda3/envs/literature/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 9.0.
INFO:fbprophet.forecaster:Disabling weekly seaso

Rates
------

The rate is calculated as the ratio of number of total articles and the number of years of publication.
Thus:
    $$ \text{rate}= \frac{\text{number of articles}}{\text{years of publication}}$$

In [14]:
(yrl.values.sum() / yrl.count())

21.166666666666668

In [15]:
rates = [np.round(yrl.values.sum() / yrl.count(), 3)]
for api in ['IEEE', 'Nature', 'PLOS', 'Springer', 'arXiv']:
    rates.append(np.round(yrl_pvn[api].values.sum() / yrl.count(), 3))

In [16]:
rates_table = pd.DataFrame(rates, ['Overall', 'IEEE', 'Nature', 'PLOS', 'Springer', 'arXiv'], columns={'rates'}).to_latex()

In [17]:
file = open("/home/nightwing/rsc/Literature-Article/assets/rates_table.tex", 'w')
file.write('{}'.format(rates_table))
file.close()